In [1]:
import os
from openai import OpenAI

# 환경 변수에서 API 키 가져오기
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

if OPENAI_API_KEY:
    print(f"API Key: {OPENAI_API_KEY}")
else:
    print("API Key가 설정되지 않았습니다.")

API Key: sk-proj-xyUeL6LCSZDpdAaJmExbT3BlbkFJo8tanm2wx5ZEnHavJC53


In [2]:
import os
from openai import OpenAI

# 환경 변수에서 API 키 가져오기
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


## 데이터 전처리 함수
* 크롤링한 데이터를 전처리

In [3]:
import json

with open('./res/reviews.json', 'r', encoding='utf-8') as f:
    review_list = json.load(f)

review_list[:5]

[{'review': '깨끗하고 너무 좋았어요', 'stars': 5, 'date': '22시간 전'},
 {'review': '위치도 아주좋고 숙소도 깨끗하고 좋아요~', 'stars': 5, 'date': '2일 전'},
 {'review': '잘쉬고놀다갑니다  감사', 'stars': 5, 'date': '6일 전'},
 {'review': '숙소는 말할 필요 없을것 같고 주변 인사동과 조계사 위로 경복궁을 도보로 품은 입지가 최고인 곳',
  'stars': 5,
  'date': '2024.05.23'},
 {'review': '3번째 방문인데 좋습니다! 주변관광지와 교통도 좋습니다.',
  'stars': 5,
  'date': '2024.05.23'}]

Good Review, Bad Review
* Good Review = 5 Stars
* Bad Review = less than 4 Stars

In [4]:
good, bad = 0, 0

for r in review_list:
    if r['stars'] == 5:
        good += 1
    else:
         bad += 1

good, bad

(177, 43)

In [5]:
reviews_good, reviews_bad = [], []

for r in review_list:
    if r['stars'] == 5: # special token 'REVIEW_START', 'REVIEW_END'
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_bad[:3]

['[REVIEW_START]인사동, 북촌동, 종각 등등 근처에 갈곳두 많구\r\n숙소두 깨끗하구 너무 좋아요\r\n엘베를 2번 타야하는게 살짝 번거롭지만\r\n그거빼구 다 좋아요[REVIEW_END]',
 '[REVIEW_START]근처 볼거리 놀거리가 가까이 있어 편했으나 \n조계사 고층뷰는 의미 없이 배정해 주었으며\n객실 내 청결상태가 별로였음 욕조 머리카락, 수건의 먼지등\n재방문 의사 없음[REVIEW_END]',
 '[REVIEW_START]2월21일(수) 오전7시경, 카운터에 계시던 남자 직원 불친절\n방음 최악[REVIEW_END]']

In [6]:
# LIST -> TEXT로 바꿔주기
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)
reviews_bad_text

'[REVIEW_START]인사동, 북촌동, 종각 등등 근처에 갈곳두 많구\r\n숙소두 깨끗하구 너무 좋아요\r\n엘베를 2번 타야하는게 살짝 번거롭지만\r\n그거빼구 다 좋아요[REVIEW_END]\n[REVIEW_START]근처 볼거리 놀거리가 가까이 있어 편했으나 \n조계사 고층뷰는 의미 없이 배정해 주었으며\n객실 내 청결상태가 별로였음 욕조 머리카락, 수건의 먼지등\n재방문 의사 없음[REVIEW_END]\n[REVIEW_START]2월21일(수) 오전7시경, 카운터에 계시던 남자 직원 불친절\n방음 최악[REVIEW_END]\n[REVIEW_START]이번방은 수압이좀 약했고\n여러번 이용했지만 이런적없었는데\n침구에 머리카락이있었고\n수건에서 냄새가났어요ㅠ\n개선해주셔야할것같아요\n전화하려다가 너무 피곤하고 힘들어서 그냥쉬다왔습니다[REVIEW_END]\n[REVIEW_START]근처에 놀 수 있는 곳이 많아서 좋았어요[REVIEW_END]\n[REVIEW_START]방음이 아예 안됩니다. 기껏 비싼돈주고 호텔왔는데 같은 층에 단체손님들인지 뭔지 새벽 5시까지 떠들고 쿵쿵치고 소리지르고 난리를 쳐서 한숨도 못잤습니다. 직원분들은 친절하셨지만 이렇게 하나도 방음이안되는 호텔은 처음이네요. 돈이 너무너무아깝고 화가납니다. 직원분들의 잘못은 아니지만, 단체손님이 들어와있으면 그 사이에 방을 배정하지않고 다른 층으로해주는 유도리가있었으면 참 좋았겠네요. 유감입니다. 두번다시 이 브랜드 호텔 이용안합니다.[REVIEW_END]\n[REVIEW_START]서비스 : 친절합니다. 리셉션에 요구사항도 빠르게 들어주셨어요. 체크인 체크아웃 또한 무리없이 빠르게 진행됩니다. \n숙소 청결도 : 바닥이나 욕조에 머리카락이 너무 많습니다. 어메니티도 아쉽습니다. 바디로션만 따로 있다거나 샤워스펀지가 없습니다. \n시설 : 방이 좁긴하지만 동선에 따른 구조는 괜찮습니다. 조계사뷰는 그닥 특별하지 않습니다. 같은건물에 음식점이나 편의점 등 상점들이 가까이 있어 편리합니다.

전처리 함수 작성

In [7]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    # json 불러오기
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []
    
    current_date = datetime.datetime.now() # 현재 날짜
    date_boundary = current_date - datetime.timedelta(days=6*30) # 6 month

    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError): # 6시간 전, 22시간 전 등 에러
            review_date = current_date # 지금 날짜

        # 리뷰 날짜가 6개월안에 속하면, 계속
        if review_date < date_boundary:
            continue

        if r['stars'] == 5: # special token 'REVIEW_START', 'REVIEW_END'
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    # LIST -> TEXT로 바꿔주기
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

good, bad = preprocess_reviews()
good[:100]

'[REVIEW_START]깨끗하고 너무 좋았어요[REVIEW_END]\n[REVIEW_START]위치도 아주좋고 숙소도 깨끗하고 좋아요~[REVIEW_END]\n[REVIEW_STAR'

## 평가용 함수
평가 기준 설정
* MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
* 비교하는 방식 VS 점수 매기는 방식
* 비교가 상대적으로 정확

평가 스크립트
* MT-Bench 논문 Prompt에서 일부 단어만 수정

In [8]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion

## Baseline 모델 개발
* 기본적인 성능을 제공하는 간단한 모델

In [9]:
prompt_baseline = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘"""

In [10]:
reviews, _ = preprocess_reviews(path='./res/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

print(summarize(reviews, prompt_baseline).choices[0].message.content)

다양한 리뷰 중에서는 숙소가 깨끗하고 위치가 좋다는 내용이 많이 언급되었습니다. 또한 시설과 서비스에 대한 만족도가 높았고, 교통편이 좋다는 평가도 많이 받았습니다.


## Baseline 모델 평가

In [11]:
# 실제 야놀자 리뷰 요약
# 고도화 작업을 통화 결과물
summary_real_20240526 = '위치가 매우 우수한 숙박시설로, 인사동과 조계사, 경복궁 등 관광지에 도보로 이동할 수 있는 편리한 위치에 있습니다. 객실은 깔끔하며 직원들의 친절한 서비스와 청결한 시설이 인상적입니다. 주변에는 맛집과 편의시설이 많아 편리하며, 교통 접근성도 좋습니다. 전체적으로 만족도가 높고 자주 방문하고 싶은 곳으로 손꼽히는 숙소로 평가됩니다.'

In [12]:
print(pairwise_eval(reviews, summarize(reviews, prompt_baseline).choices[0].message.content, summary_real_20240526).choices[0].message.content)

Assistant A’s summary is brief and general, stating that most reviews mention cleanliness and satisfaction with the accommodation. It highlights the common themes of cleanliness and good location but lacks depth and specific details.

Assistant B’s summary, on the other hand, provides a more detailed and comprehensive overview. It mentions specific attractions like 인사동, 조계사, and 경복궁, and highlights the convenience of the location. It also notes the cleanliness of the rooms, the friendliness of the staff, and the abundance of nearby amenities. This summary captures a broader range of positive aspects mentioned in the reviews and provides a clearer picture of why the accommodation is well-regarded.

Therefore, Assistant B’s response is more helpful, relevant, accurate, and detailed.

[[B]]


## 대규모 평가 스크립트
* 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [13]:
# 10개의 다른 리뷰 요약을 해줘
eval_count = 10

summaries_baseline = [summarize(reviews, prompt_baseline, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

['다양한 리뷰가 있지만 대체로 깨끗하고 위치가 좋다고 평가한 것 같습니다. 강추할 만한 숙소라는 인상을 받았어요.',
 '다양한 리뷰가 있지만 대부분이 깨끗하고 위치가 좋다고 하며 전반적으로 만족한 숙소라는 내용을 담고 있다.',
 '다양한 리뷰가 있지만 대부분 깨끗하고 위치가 좋고 만족하며 편안한 숙박을 즐긴다고 한다.',
 '다양한 리뷰들 중에서는, "깨끗하고 너무 좋았어요"라는 리뷰가 돋보였습니다.',
 '다양한 리뷰가 있지만 대부분이 깔끔하고 위치가 좋다는 내용을 담고 있습니다. 숙소 시설이나 서비스에 대한 긍정적인 평가도 많이 보입니다.',
 '다양한 리뷰 중 가장 일반적인 내용인 "깔끔하고 위치가 좋았음"으로 요약할 수 있겠습니다.',
 '숙소는 깨끗하고 좋은 위치에 있어서 만족했다.',
 '다양한 손님들로부터 깨끗하고 좋은 위치에 위치한 숙소에 대한 긍정적 리뷰가 많이 있습니다.',
 '다양한 리뷰가 있지만 대체적으로 깨끗하고 좋은 위치에 위치한 숙소라는 내용이 강조된다.',
 '모든 리뷰가 깨끗하고 좋았다는 내용을 강조하고 있으며, 위치와 시설에 대한 만족도가 높은 편이다. 각 리뷰마다 특정한 호감 표현이나 불만 사항이 나타나는 것은 없는 것으로 보인다.']

In [14]:
# 대규모 평가 스크립트
from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content

        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else:
            print('Evaluation Error')
    return a_cnt, b_cnt, draw_cnt

wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20240526 for _ in range(len(summaries_baseline))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:48<00:00,  4.90s/it]

Wins: 0, Losses: 10, Ties: 0


* 평가를 10번하는건 충분하지 않지만 일단 진행 (50~100)
* 10번 평가 중 2번은 좋은 평가를 받음

## 모델 고도화 1 - 조건 명시

In [15]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count=10
# 리뷰 요약
summarise = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]

# 평가
wins, losses, ties = pairwise_eval_batch(reviews, summarise, [summary_real_20240526 for _ in range(len(summarise))])
print(f'Wins: {wins}, losses: {losses}, ties: {ties}')

100%|██████████| 10/10 [01:06<00:00,  6.69s/it]

Wins: 0, losses: 10, ties: 0


## 모델 고도화 2 - 입력 데이터 품질 증가

In [16]:
# 전처리 함수 수정
# 고품질 리뷰들만 남기자
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    # json 불러오기
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []
    
    current_date = datetime.datetime.now() # 현재 날짜
    date_boundary = current_date - datetime.timedelta(days=6*30) # 6 month

    filtered_cnt = 0
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError): # 6시간 전, 22시간 전 등 에러
            review_date = current_date # 지금 날짜

        # 리뷰 날짜가 6개월안에 속하면, 계속
        if review_date < date_boundary:
            continue
        # 고도화 2, 리뷰 길이 30미만 필터링
        if len(r['review']) < 30:
            filtered_cnt += 1
            continue

        if r['stars'] == 5: # special token 'REVIEW_START', 'REVIEW_END'
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    # LIST -> TEXT로 바꿔주기
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()

* 리뷰 길이가 30미만인 리뷰만 출력
* 40개가 필터링 됨
* 좋은 리뷰는 23개

In [17]:
eval_count = 10
summarise = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summarise, [summary_real_20240526 for _ in range(len(summarise))])
print(f'Wins: {wins}, losses: {losses}, ties: {ties}')

100%|██████████| 10/10 [01:26<00:00,  8.69s/it]

Wins: 8, losses: 2, ties: 0


* 모델 고도화라는 말이 있지만, 어떤 데이터를 쓰느냐 중요! 입력데이터의 중요성! (전처리 강조)
* 길이를 <30으로 하였으나, 더 자르면 좋아질 수 있다.

## 모델 고도화 3 - Few-Shot Prompting

In [18]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summarise = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summarise, [summary_real_20240526 for _ in range(len(summarise))])
print(f'Wins: {wins}, losses: {losses}, ties: {ties}')

100%|██████████| 10/10 [01:21<00:00,  8.14s/it]

Wins: 6, losses: 4, ties: 0


In [19]:
summarise

['이 숙소는 말할 필요 없을 것 같고 주변 인사동과 조계사 위로 경복궁을 도보로 품은 입지가 최고입니다.\n\n3번째 방문인데 좋습니다! 주변 관광지와 교통도 좋습니다.\n\n나인트리인사동은 서울 갈 때마다 하루는 꼭 숙박하는 곳입니다. 시설이나 서비스 위치, 뭐 하나 빠지는 것 없는 최고의 숙소입니다. 이번에도 정말 편안하게 잘 쉬다 왔습니다. ^^\n\n엄마랑 같이 이용했는데 위치도 좋고 깔끔하고 좋았습니다.\n\n시설도 깨끗하고 호텔 문을 나오면 인사동이라 주변 관광하기 너무 편리합니다 ^^',
 '이 숙소는 인사동과 조계사 위로 경복궁을 품은 입지가 최고입니다. 주변 관광지와 교통이 편리하여 관광하기 좋고, 시설이나 서비스에 만족한 손님이 많습니다. 또한 위치가 좋아서 다음 서울 여행 때도 꼭 묵고 싶어하는 손님이 많습니다. 종합적으로 나인트리인사동은 시설도 좋고 서비스도 좋아서 다시 방문하고 싶은 숙소입니다.',
 '이 숙소는 말할 필요 없이 주변 인사동과 조계사 위로 경복궁을 도보로 품은 입지가 최고입니다. 주변관광지와 교통이 좋아서 3번째 방문인데 좋았습니다. 시설과 서비스 등 뭐 하나 빠지는 것 없이 편안하고 깔끔한 숙소입니다. 위치와 교통이 최고의 장점이며 다음 서울 여행에도 묵고 싶은 곳이었습니다. 나인트리 최고의 숙소 중 하나입니다.',
 '이 숙소는 주변 관광지와 교통이 편리한 곳에 위치하고 있습니다. 시설이나 서비스는 빠른 것 없이 편안한 휴식을 제공해주며, 위치가 인사동 중심이라 접근성이 용이합니다. 즐거운 서울 여행을 위해 다시 방문하고 싶은 곳입니다. 또한, 깨끗한 시설과 친절한 직원들이 좋은 인상을 줍니다. 늘 머물고 싶은 곳 중 하나입니다.',
 '이 숙소는 말할 필요 없을 정도로 인사동과 조계사, 경복궁 등 주변 관광지를 도보로 품은 최고의 입지를 자랑합니다. 이전 방문자들은 주변 교통과 관광지에 대해 좋은 평가를 내리고 있습니다. 또한, 시설과 서비스, 위치 등 어느 부분에서도 부족함이 없는 최고의 숙소로 손꼽히고 있습니다

In [20]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summarise = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summarise, [summary_real_20240526 for _ in range(len(summarise))])
print(f'Wins: {wins}, losses: {losses}, ties: {ties}')

100%|██████████| 10/10 [01:30<00:00,  9.07s/it]

Wins: 6, losses: 4, ties: 0


In [ ]:
summaries

In [22]:
reviews_2shot, _ = preprocess_reviews('./res/ninetree_yongsan.json')
summary_2shot = summarize(reviews_2shot, prompt_1shot, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_2shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [01:15<00:00,  7.53s/it]

Wins: 9, Losses: 1, Ties: 0


In [23]:
summaries

['이 숙소는 인사동과 주요 관광지에 위치한 우수한 숙소로 알려져 있습니다. 방은 깨끗하고 시설은 훌륭하며, 직원들의 친절함과 서비스도 만족스러운 편입니다. 주변 교통의 편리함과 맛집 다수로 둘러싸인 이 숙소는 다음 방문을 고려할 만한 높은 만족도를 자랑합니다.특히 인사동과 근접한 정확한 위치는 숙소 이용객들에게 편의를 제공하고 있습니다.하지만 일부 이용객들은 복도에서 소음이 발생하는 것과 세면대 청결에 미흡함을 지적했습니다. 이러한 단점을 보완하면 최고의 숙박 체험을 제공할 수 있을 것으로 보입니다.',
 '이 숙소는 인사동과 근접한 위치에 있어 관광이나 쇼핑을 즐기기에 아주 편리합니다. 주변 교통이 편리하고 청결한 시설을 제공하며, 직원들도 친절한 편으로 알려져 있습니다. 특히 위치와 접근성이 좋은 것으로 평가받고 있고, 서울 여행 시 추천할만한 가격대와 서비스를 제공하는 것으로 유명합니다. 하지만 몇몇 리뷰에선 청결 상태와 프론트 데스크 서비스에 대한 개선점이 언급되었습니다.다른 리뷰들에서는 위치나 근접한 맛집을 즐기거나 교통편이 좋았다는 긍정적인 평가가 많이 나왔습니다.',
 '이 숙소는 교통의 요지에 위치하여 주변 관광지와 이동이 편리하고, 시설과 서비스 모두 만족스러워하는 손님들이 많이 찾는 곳으로 평가받고 있습니다. 위치가 접근성이 좋고 깔끔한 객실은 편안한 휴식을 즐길 수 있도록 도와줍니다. 또한, 직원의 친절함과 시설의 청결함이 긍정적인 인상을 남깁니다. 하지만 일부 손님들은 프론트 직원의 서비스에 조금 불만을 표현하고 있습니다. 전반적으로는 이 호텔을 다음 방문을 희망하는 많은 손님들의 호평을 받고 있습니다.',
 '요약:\n나인트리 인사동은 교통이 편리하고 주변 인사동, 조계사, 경복궁을 도보로 방문하기에 좋은 위치에 위치한 호텔이다. 호텔은 깔끔하고 시설과 서비스가 훌륭하며, 주변 관광지와 교통편을 누리기에 편리하다는 이용객들의 호평을 받고 있다. 특히 인사동 한복판이라는 접근성은 더욱 매력적으로 다가오는 것으로 보인다. 전체적으로 고객들